# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [368]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [369]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [370]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [371]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [372]:
import pandas as pd
from math import floor

In [373]:
document = ET.parse( './data/mondial_database.xml' )

## 1. 10 countries with the lowest infant mortality rates

The approach here is to go through each country and 

In [374]:
def try_country_subelement(element, sub_el_name):
    '''
    element: xml.etree.ElementTree.Element to be searched through
    sub_el_name: string of subelement name to be found
    If found: return list with country name and subelement text property
    If not  : return list with country name and none
    '''
    try:
        sub_element = [element.find('name').text, element.find(sub_el_name)]
    except:
        sub_element = [element.find('name').text, None]
    
    return sub_element

In [375]:
infant_mortality_rate_dict = {}
for element in document.iterfind('country'):
    country_imr = try_country_subelement(element,'infant_mortality')
    if country_imr[1] is not None:
        infant_mortality_rate_dict[country_imr[0]] = float(country_imr[1].text)
    else:
        infant_mortality_rate_dict[country_imr[0]] = country_imr[1]


In [376]:
df_imr = pd.DataFrame.from_dict(infant_mortality_rate_dict,orient='index').reset_index()
df_imr.columns=['country','infant_mortality_rate']
top_10_imr = df_imr.sort_values('infant_mortality_rate',ascending=True).head(10)
top_10_imr.index = range(1,11)

In [377]:
#Top 10 Countries with lowest Infant Mortality Rates
top_10_imr

,country,infant_mortality_rate
1,Monaco,1.81
2,Japan,2.13
3,Bermuda,2.48
4,Norway,2.48
5,Singapore,2.53
6,Sweden,2.60
7,Czech Republic,2.63
8,Hong Kong,2.73
9,Macao,3.13
10,Iceland,3.15


## 2. 10 cities with the largest population
The approach here is to go through each country and find all listed population data. Once found, only pick out the latest year's data and compile it into a dictionary containing the country, city, and population value.

In [378]:
# Create a list to hold dicts containing {country, city, population}
city_populations = []

# Extract population information
for element in document.iterfind('country'):
    for city in element.iterfind('city'):
        # Create a dict to store all poplulation data found for a city
        pop_dict = {}
        for pop in city.iterfind('population'):
            pop_dict[int(pop.attrib['year'])] = int(pop.text)
        # Pick the city population value with the latest year
        latest_population_count = pop_dict[max(pop_dict.keys())] if pop_dict else None
        # Add a dict to our 'city_populations' list containing the country, city, and population value
        city_populations.append({'country':element.find('name').text,'city':city.find('name').text,'population':latest_population_count})

In [379]:
# convert list of dicts to DataFrame
df_city_pop = pd.DataFrame.from_records(city_populations) 
# reorder columns for clarity
df_city_pop = df_city_pop[['country','city','population']] 

# sort by population and make a top 10 df using '.head(10)'
top_10_city_populations = df_city_pop.sort_values('population', ascending=False).head(10) 
# reindex top 10 df to show index as ranking from 1-10
top_10_city_populations.index = range(1,11)

In [380]:
top_10_city_populations

,country,city,population
1,South Korea,Seoul,9708483
2,Egypt,Al Qahirah,8471859
3,Thailand,Bangkok,7506700
4,Hong Kong,Hong Kong,7055071
5,Vietnam,Ho Chi Minh,5968384
6,Singapore,Singapore,5076700
7,Egypt,Al Iskandariyah,4123869
8,Taiwan,New Taipei,3939305
9,South Korea,Busan,3403135
10,North Korea,Pyongyang,3255288


## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

The approach here is to go through each country and find two things:
1. Latest Country Population
2. All the ethnic groups

Each ethnic group has a 'percentage' attribute that indicates what percentage of the population in that country is made up of that ethnic group. After determining current population value for a country we can multiply those two numbers together to determine the total ethnic population in that country. We then add this determined population value to the total population tally for that ethnic group. This will let us determine which ethnic groups have the largest overall populations.

In [381]:
# Create a dictionary containing {ethnicgroup, population}
ethnicgroup_populations = {}

# Find ethnic groups per country and calculate their population
for country in document.iterfind('country'):    
    
    # Find latest country population
    pop_dict = {}
    for pop in country.iterfind('population'):
        pop_dict[int(pop.attrib['year'])] = int(pop.text)
    country_population = pop_dict[max(pop_dict.keys())] if pop_dict else None
    
    # Find all ethnic groups in a country and compute ethnic population value using 
    # 'country_population' and 'percentage' attribute associated with ethnic group
    for egroup in country.iterfind('ethnicgroup'):
        egroup_percent = float(egroup.attrib['percentage'])*.01
        ethnic_population = floor(country_population * egroup_percent)
        
        # Store this computed population in 'ethnicgroup_populations' dictionary
        if egroup.text in ethnicgroup_populations.keys():
            ethnicgroup_populations[egroup.text] += ethnic_population
        else:
            ethnicgroup_populations[egroup.text] = ethnic_population


In [382]:
# Create a DataFrame using the 'ethnicgroup_populations' dictionary
df_ethnic_populations = pd.DataFrame.from_dict(ethnicgroup_populations,orient='index').reset_index()
# Rename columns for clarity
df_ethnic_populations.columns = ['ethnicgroup','population']
# Sort the DataFrame by 'population'
df_ethnic_populations.sort_values('population',ascending=False,inplace=True)

# Create a top 10 DataFrame and adjust index to match ranking
top_10_ethnic_groups = df_ethnic_populations.head(10)
top_10_ethnic_groups.index = range(1,11)

In [383]:
top_10_ethnic_groups

,ethnicgroup,population
1,Han Chinese,1245058800
2,Indo-Aryan,871815583
3,European,494872201
4,African,318325104
5,Dravidian,302713744
6,Mestizo,157734349
7,Bengali,146776916
8,Russian,131856989
9,Japanese,126534212
10,Malay,121993548


## 4. Name and country of a) longest river, b) largest lake and c) airport at highest elevation

### A)

In [384]:
countries = {}
for country in document.iterfind('country'):
    countries[country.attrib['car_code']] = country.find('name').text

In [385]:
rivers = []
for river in document.iterfind('river'):
    name = river.find('name').text
    # Determine Location
    if river.find('located') is not None:
        country = countries[river.find('located').attrib['country']]
    else:
        country = countries[river.find('source').attrib['country']]
    
    # Determine Length
    if river.find('length') is not None:
        length = float(river.find('length').text)
    else:
        length = None
    
    # Add dictionary of answers to 'rivers' list
    rivers.append({
            'name':name,
            'country':country,
            'length':length
        })

In [386]:
df_rivers = pd.DataFrame.from_records(rivers).sort_values('length',ascending=False).reset_index(drop=True)
longest_river = df_rivers.iloc[0]
longest_river

country    Colombia
length         6448
name       Amazonas
Name: 0, dtype: object

### B)

In [387]:
lakes = []
for lake in document.iterfind('lake'):
    name = lake.find('name').text
    # Determine Location
    if lake.find('located') is not None:
        country = countries[lake.find('located').attrib['country']]
    else:
        country = countries[lake.attrib['country'].split(' ')[0]]
    
    # Determine size
    if lake.find('area') is not None:
        area = float(lake.find('area').text)
    else:
        area = None
    
    # Add dictionary of answers to 'lakes' list
    lakes.append({
            'name':name,
            'country':country,
            'area':area
        })

In [388]:
df_lakes = pd.DataFrame.from_records(lakes).sort_values('area',ascending=False).reset_index(drop=True)
largest_lake = df_lakes.iloc[0]
largest_lake

area            386400
country         Russia
name       Caspian Sea
Name: 0, dtype: object

### C)

In [389]:
airports = []
for airport in document.iterfind('airport'):
    name = airport.find('name').text
    country = countries[airport.attrib['country']]
    
    # Determine elevation
    if airport.find('elevation').text is not None:
        elevation = float(airport.find('elevation').text)
    else:
        elevation = None
        
    # Add dictionary of answers to 'airports' list
    airports.append({
            'name':name,
            'country':country,
            'elevation':elevation
        })

In [390]:
df_airports = pd.DataFrame.from_records(airports).sort_values('elevation',ascending=False).reset_index(drop=True)
highest_airport = df_airports.iloc[0]
highest_airport

country           Bolivia
elevation            4063
name         El Alto Intl
Name: 0, dtype: object

***
## Answers


### 1) 10 countries with the lowest infant mortality rates

In [391]:
top_10_imr

,country,infant_mortality_rate
1,Monaco,1.81
2,Japan,2.13
3,Bermuda,2.48
4,Norway,2.48
5,Singapore,2.53
6,Sweden,2.60
7,Czech Republic,2.63
8,Hong Kong,2.73
9,Macao,3.13
10,Iceland,3.15


### 2) 10 cities with the largest population

In [392]:
top_10_city_populations

,country,city,population
1,South Korea,Seoul,9708483
2,Egypt,Al Qahirah,8471859
3,Thailand,Bangkok,7506700
4,Hong Kong,Hong Kong,7055071
5,Vietnam,Ho Chi Minh,5968384
6,Singapore,Singapore,5076700
7,Egypt,Al Iskandariyah,4123869
8,Taiwan,New Taipei,3939305
9,South Korea,Busan,3403135
10,North Korea,Pyongyang,3255288


### 3) 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [393]:
top_10_ethnic_groups

,ethnicgroup,population
1,Han Chinese,1245058800
2,Indo-Aryan,871815583
3,European,494872201
4,African,318325104
5,Dravidian,302713744
6,Mestizo,157734349
7,Bengali,146776916
8,Russian,131856989
9,Japanese,126534212
10,Malay,121993548


### 4 A) Name and country of longest river
* Name: Amazonas
* Country: Columbia
* Length: 6448

In [394]:
longest_river

country    Colombia
length         6448
name       Amazonas
Name: 0, dtype: object

### 4 B) Name and country of largest lake
* Name: Caspian Sea
* Country: Russia
* Area: 386400

In [395]:
largest_lake

area            386400
country         Russia
name       Caspian Sea
Name: 0, dtype: object

### 4 C) Name and country of airport at highest elevation
* Name: El Alto Intl
* Country: Bolivia
* Elevation: 4063

In [396]:
highest_airport

country           Bolivia
elevation            4063
name         El Alto Intl
Name: 0, dtype: object